In [1]:
import os

import numpy as np
import pandas as pd
import cv2

In [5]:
image_filepaths = []

for root, dirs, files in os.walk('./analysis/mask'):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_filepaths.append(os.path.join(root, file))

In [8]:
df = pd.DataFrame([], columns=['Date', 'DPI', 'Tube', 'Level', 'Path'])

for index, filepath in enumerate(image_filepaths):
    filename_parts = filepath.split('/')
    date = filename_parts[3].split('_')[0]
    dpi = filename_parts[3].split('_')[2].removesuffix('dpi')
    tube = filename_parts[4].split('_')[1].removeprefix('T')
    level = filename_parts[4].split('_')[2].removeprefix('L').removesuffix('.PNG')
    df.loc[index] = ([date, dpi, tube, level, filepath])

df['Date'] = pd.to_datetime(df['Date'], format='%m%d%Y')
df['DPI'] = df['DPI'].astype(int)
df['Tube'] = df['Tube'].astype(int)
df['Level'] = df['Level'].astype(int)

df.sort_values(by=['Date', 'Tube', 'Level'], inplace=True)

In [9]:
for name, group in df.groupby(['Date', 'Tube']):
    images = [cv2.imread(row['Path']) for index, row in group.iterrows()]
    min_level = group['Level'].min()
    max_level = group['Level'].max()
    for index in range(0, len(images) - 1):
        if group['DPI'].mean() == 100:
            images[index] = images[index][:, :-50, :]
        elif group['DPI'].mean() == 300:
            images[index] = images[index][:, :, :]
    
    image = np.concatenate(images, axis=1)

    if not os.path.exists(f'./mrphotos/output/{name[0].strftime("%m%d%Y")}'):
        os.makedirs(f'./mrphotos/output/{name[0].strftime("%m%d%Y")}')
    cv2.imwrite(f'./mrphotos/output/{name[0].strftime("%m%d%Y")}/CS_T{name[1]}_L{min_level}-{max_level}.png', image)